<a href="https://colab.research.google.com/github/anurag0308/HRanalytics/blob/master/HR_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf 
tf.test.gpu_device_name() 

'/device:GPU:0'

In [13]:
! pip install pandas 


In [14]:
from google.colab import drive 
drive.mount('/mntDrive') 


Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [15]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/train_LZdllcl.csv')

FileNotFoundError: ignored